# SECTION 07: SQL DATABASES

## LEARNING OBJECTIVES:

- Understand what a database is and how it is different than a DataFrame/Excel sheet.
- Understand how to read database map
    - Primary keys vs forgein keys
- Understand how to select, filter, order, and group data using SQL
- Understand the different types of Joins 


- Breakout Group Activity:  [Survive on sql-island](https://sql-island.informatik.uni-kl.de/)

## Questions:

- Can you explain the below solution when trying to find offices with 0 employees?

```python
cur.execute("""SELECT o.officeCode, o.city, COUNT(e.employeeNumber) AS n_employees
               FROM offices AS o 
               LEFT JOIN employees AS e
               USING(officeCode)
               GROUP BY officeCode
               HAVING n_employees = 0;""")
```

- What is the difference between sqlite.connect() and sqlite.Connection()?

- Subqueries Lab section 1.8 Can we go over this solution?


# SQL Databases

<img src="https://raw.githubusercontent.com/learn-co-students/dsc-sql-introduction-online-ds-sp-000/master/images/Database-Schema.png">


- SQL is designed to work with **relational data**. 
- This really just means pieces of data that are **related to eachother**.

- Each table has a **primary key** (like a DataFrame index), with a unique index for each row in the database.
- The name of the primary key is preceded by an asterix (\*). 

- Columns that are the **primary key one on table** can also appear on **other tables**. 
    - Then it is refered to as a **foreign key** aka the primary key from a different ("foreign") table. 

### ⨠ Q: Why do we need databases? Why can't we just use a bunch of Pandas DataFrames?

- 

## Querying Databases - `SELECT`ing data



- To retrieve data from one or more tables you usually use a `SELECT` statement. 
```SQL
SELECT * FROM table;
```


> - NOTE: SQL queries dot not _have_ to be all-caps, but it is a convention to help differentiate sql syntax versus names of tables/columns.



- A more advanced select query.
```SQL
SELECT col1, col2, col3
FROM table
WHERE records match criteria
LIMIT 100;
```

- **All select statements must:**
    1. **start with the `SELECT`**
    2. followed by **what you want to select**. Separate multiple column names separated by a `,` 
    3. Then specify where the data is coming `FROM` followed by the table name. 
    4. **Afterward, you can provide conditions such as filters or sorting**.

```SQL
SELECT *
FROM payments
ORDER BY amount DESC
LIMIT 10;
```



## SQL with `sqlite3`

- Use `sqlite3` for SQL queries in Python.
    1. Connect to database
    2. Create a cursor.
    3. Form your query
    4. Execute/fetch your results.

```python
import sqlite3
connection = sqlite3.connect('pet_database.db') # Creates pet_database, but empty until create a table    
cursor = connection.cursor()


# Select from table
cursor.execute('''SELECT name FROM cats;''').fetchall()

```

In [1]:
# !pip install -U fsds
# from fsds.imports import *
import sqlite3
import pandas as pd
import os,glob,sys
os.getcwd()



'/Users/jamesirving/Documents/GitHub/_STUDY GROUP PREP/online-dtsc-pt-041320-cohort-notes/Instructor Notebooks'

In [4]:
db = '../datasets/SQL/data.sqlite'

In [5]:
# connect to database
conn = sqlite3.connect(db)
cur = conn.cursor()

In [6]:
# sqlite3.connect(db)
# sqlite3.Connection()

### How to get all of the table names in a database

- The container for all tables in a database with sqlite3 is called `sqlite_master` 
- We can find the name of all of the tables in a db using:
```python
table_names = cur.execute("""
SELECT name 
FROM sqlite_master 
WHERE type='table';
""").fetchall()
```

In [ ]:
# Get table names
table_names = cur.execute("""
SELECT name 
FROM sqlite_master 
WHERE type='table';
""").fetchall()
table_names

In [ ]:
table_names = [x[0] for x in table_names]
table_names

<img src="https://raw.githubusercontent.com/learn-co-students/dsc-sql-introduction-online-ds-sp-000/master/images/Database-Schema.png" width=500>

### How to get the column names after executing a query:


In [ ]:
df = pd.DataFrame(cur.execute("select * from products").fetchall())
df

- the cursor has a `.description` that contains information about the column names

In [ ]:
cur.description

In [ ]:
col_names =[col[0] for col in cur.description]
print(col_names)

In [ ]:
df = pd.DataFrame(cur.execute('select * from products').fetchall(),
                  columns=col_names)
df.head()

# FILTERING AND ORDERING

- `ORDER BY` - `DESC`/`ASC`
- `LIMIT`
- `BETWEEN`
- `NULL`
- `COUNT`
- `GROUP BY`

In [ ]:
query = """select * from products
GROUP BY productLine 
ORDER BY quantityInStock DESC;"""
data = cur.execute(query).fetchall()
col_names =[col[0] for col in cur.description]
df = pd.DataFrame(data,
                  columns=col_names)
df

## GROUPING DATA WITH SQL

- Like we do with Pandas, we can use GROUP BY statements in SQL and then apply **aggregate functions:**
    - `COUNT`
    - `MAX`
    - `MIN`
    - `SUM`
    - `AVG`

In [ ]:
cur.execute("""SELECT city, COUNT(employeeNumber)
FROM offices 
JOIN employees
USING(officeCode)
GROUP BY city
ORDER BY count(employeeNumber) DESC;""")

df = pd.DataFrame(cur.fetchall())

df.columns = [x[0] for x in cur.description]

df.head()

## ALIASING

- can assign a temporary name to data being imported
- Useful for `JOIN`,`GROUP BY`, and aggregates.

In [ ]:
cur.execute("""SELECT city, COUNT(employeeNumber) AS numEmployees
               FROM offices
               JOIN employees
               USING(officeCode)
               GROUP BY 1
               ORDER BY numEmployees DESC;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

In [ ]:
cur.execute("""SELECT customerName,
               COUNT(customerName) AS number_purchases,
               MIN(amount) AS min_purchase,
               MAX(amount) AS max_purchase,
               AVG(amount) AS avg_purchase,
               SUM(amount) AS total_spent
               FROM customers
               JOIN payments
               USING(customerNumber)
               GROUP BY 1
               ORDER BY SUM(amount) DESC;""")
df = pd.DataFrame(cur.fetchall())
df. columns = [i[0] for i in cur.description]
print(len(df))
df.head()

## The `WHERE` Clause

In general, the `WHERE` clause filters query results by some condition. As you are starting to see, you can also combine multiple conditions.

- 
```python
cur.execute("""SELECT * FROM customers WHERE city = 'Boston' OR city = 'Madrid';""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df
```


- To refine your searches, you can add `ORDER BY` and `LIMIT` clauses. 
    - The order by clause allows you to sort the results by a particular feature.
- Finally, the limit clause is typically the last argument in a SQL query and simply limits the output to a set number of results.



In [ ]:
cur.execute("""SELECT * FROM customers WHERE city = 'Boston' OR city = 'Madrid';""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

## The `HAVING` clause

 The `HAVING` clause works similarly to the `WHERE` clause, except it is used to filter data selections on conditions **after** the `GROUP BY` clause.

In [ ]:
cur.execute("""SELECT city, COUNT(customerNumber) AS number_customers
               FROM customers
               GROUP BY 1
               HAVING COUNT(customerNumber)>=5;""")
df = pd.DataFrame(cur.fetchall())
df. columns = [i[0] for i in cur.description]
print(len(df))
df.head()

## Combining `WHERE` and `HAVING`

We can also use the `WHERE` and `HAVING` clauses in conjunction with each other for more complex rules.

- For example, let's say we want a list of customers who have made at least 3 purchases of over 50K each.

In [ ]:
cur.execute("""SELECT customerName,
               COUNT(amount) AS number_purchases_over_50K
               FROM customers
               JOIN payments
               USING(customerNumber)
               WHERE amount >= 50000
               GROUP BY 1
               HAVING count(amount) >= 3
               ORDER BY count(amount) DESC;""")
df = pd.DataFrame(cur.fetchall())
df. columns = [i[0] for i in cur.description]
print(len(df))
df.head()

## `JOIN` STATEMENTS

<img src="https://raw.githubusercontent.com/learn-co-students/dsc-sql-introduction-online-ds-sp-000/master/images/Database-Schema.png">

### Task: Displaying product details along with order details

Let's say you need to generate some report that includes details about products from orders. To do that, we would need to take data from multiple tables in a single statement. 

In [ ]:
# import sqlite3
# import pandas as pd
# conn = sqlite3.connect('/content/drive/My Drive/Datasets/data.sqlite')
# cur = conn.cursor()

In [ ]:
cur.execute("""SELECT * 
               FROM orderdetails
               JOIN products
               ON orderdetails.productCode = products.productCode
               LIMIT 10;
               """)
df = pd.DataFrame(cur.fetchall()) #Take results and create dataframe
df.columns = [i[0] for i in cur.description]
df.head()

## The `USING` clause

- If the column name is identical,you can use  is the `USING` clause. 
- Rather then saying on `tableA.column = tableB.column` we can simply say `using(column)`. 
- Only works if the column is **identically named** for both tables.

# TYPES OF JOINS

- Joins may be:
    - INNER (default)
    - OUTER
    - LEFT 
    - RIGHT

<img src="https://raw.githubusercontent.com/learn-co-students/dsc-join-statements-online-ds-sp-000/master/images/venn.png">


## Primary vs Foreign Keys
- primary key:
- forgein key:


### One-to-One, One-to-many, many-to-many Joins


- **Let's say we have databases A and B**


- **One-to-One joins:**
    - There is only 1 entry in database B that aligns with each individual entry in database A
    - e.g. A person and their social security number.
    
    
- **One-to-Many join:**
    - There are multiple entries in database B that match the entry in database A
    - e.g. Joining an order number from db A with the individual products in db B.
    
    
- **Many-to-many joins:**
    - There are multiple entries in database A that match multiple entries in database B.
    - e.g. A = classes at a college, B = students.

## SQL Subqueries

```python
cur.execute("""SELECT lastName, firstName, officeCode
               FROM employees
               WHERE officeCode IN (SELECT officeCode
                                    FROM offices 
                                    WHERE country = "USA");
                                    """)
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df
```

# Pandas + SQL

## SQL QUERY IN PANDAS - `df.query()

- Pandas DataFrames have a method called `.query()`
- This allows us to use SQL-like commands to reference data.
```python
## Normal Pandas Syntax
foo_df = bar_df.loc[bar_df['Col_1']>bar_df['Col_2']]
```

```python
## Using .query()
foo_df = bar_df.query("Col_1 > Col_2")
```
- How to use:
    - Enter the querty as a single string, using just column names to reference data.
    - To use and/or statements, use `&` and `|`, respectively

```python
foo_df = bar_df.query("Col_1 > Col_2 & Col_2 <= Col_3")
```

## Using SQL syntax with `pandasql`


- There is a library is called [pandasql](https://pypi.org/project/pandasql/) that allows for sql queries with pandas

We can install `pandasql` using the bash command `pip install pandasql`.

### Importing pandasql

In order to use `pandasql`, we need to start by importing a `sqldf` object from `pandasql`

```python
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())
```

### Writing Queries
```python
q = """SELECT
        m.date, m.beef, b.births
     FROM
        meats m
     INNER JOIN
        births b
           ON m.date = b.date;"""

results = pysqldf(q)

```

## SQL Data Types

- Data types in SQLite3:
    - https://www.sqlite.org/datatype3.html
    
- Data types:
    - TEXT
    - INTEGER
    - REAL
    - BLOB
    - NULL

## DATABASE ADMIN 101

- `CREATE TABLE table_name`
    -  Must include (col_name, datatype, and if its the key)
```PYTHON
  cur.execute("""CREATE TABLE cats (
    id INTEGER PRIMARY KEY,
    name TEXT,
    age INTEGER,
    breed TEXT)
    """)  
    ```
- `DROP TABLE table_name`
- `INSERT INTO table_name`
    - list the columns to fill in first and then the VALUES()   
```python
cur.execute('''INSERT INTO cats (name, age, breed) 
                  VALUES ('Maru', 3, 'Scottish Fold');
            ''')
```

In [9]:
# cur.execute("""CREATE TABLE cats (
#     id INTEGER PRIMARY KEY,
#     name TEXT,
#     age INTEGER,
#     breed TEXT)
#     """)